# MANU 465 Final Exam - Personality Predictor

### Author:

Liam Bontkes, 25530163

## Project Description

This project uses images of shoes to estimate a person's personality. The project uses CNN to

## Importing the Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

2021-12-07 18:48:13.989589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-07 18:48:13.989609: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Data Preprocessing

In [2]:
from keras.preprocessing.image import ImageDataGenerator

image_data_generator = ImageDataGenerator(rescale=1./255,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True)

training_set = image_data_generator.flow_from_directory('training_set',
                                                        target_size=(64, 64),
                                                        batch_size=32,
                                                        class_mode='categorical')

test_set = image_data_generator.flow_from_directory('test_set',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='categorical')

Found 1397 images belonging to 5 classes.
Found 381 images belonging to 5 classes.


## CNN Model

### Build the CNN Model

In [3]:
# initialize the model
cnn_model = tf.keras.models.Sequential()

2021-12-07 18:48:15.090440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-07 18:48:15.090840: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-07 18:48:15.090892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-07 18:48:15.090939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-07 18:48:15.090984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
# add and pool first layer
cnn_model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

In [5]:
# add and pool second layer
cnn_model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### Flatten the CNN Model

In [6]:
cnn_model.add(tf.keras.layers.Flatten())

### Add Connection Layer

In [7]:
cnn_model.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Add Output Layers

In [8]:
# add output node for each category
cnn_model.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))

### Compile the Model

In [9]:
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Train the Model

In [10]:
train_history = cnn_model.fit(x=training_set, validation_data=test_set, epochs=25)

2021-12-07 18:48:15.324126: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
44/44 [==============================] - 9s 205ms/step - loss: 0.4058 - accuracy: 0.7251 - val_loss: 0.2748 - val_accuracy: 0.7244
Epoch 2/25
44/44 [==============================] - 8s 180ms/step - loss: 0.1174 - accuracy: 0.8976 - val_loss: 0.1720 - val_accuracy: 0.8556
Epoch 3/25
44/44 [==============================] - 8s 181ms/step - loss: 0.1063 - accuracy: 0.9084 - val_loss: 0.2273 - val_accuracy: 0.8084
Epoch 4/25
44/44 [==============================] - 8s 189ms/step - loss: 0.0788 - accuracy: 0.9377 - val_loss: 0.2824 - val_accuracy: 0.7507
Epoch 5/25
44/44 [==============================] - 9s 202ms/step - loss: 0.0606 - accuracy: 0.9513 - val_loss: 0.1858 - val_accuracy: 0.8688
Epoch 6/25
44/44 [==============================] - 10s 231ms/step - loss: 0.0580 - accuracy: 0.9563 - val_loss: 0.2182 - val_accuracy: 0.8504
Epoch 7/25
44/44 [==============================] - 8s 189ms/step - loss: 0.0479 - accuracy: 0.9635 - val_loss: 0.1857 - val_accuracy: 0.8766
Epoch

KeyboardInterrupt: 

In [ ]:
# plot the accuracy and loss

performance = pd.DataFrame(train_history.history)

performance.plot()
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.title("CNN Model Training")
plt.show()

## Make a Prediction

In [ ]:
from keras.preprocessing import image

prediction_image = image.load_img('single_prediction.jpg', target_size=(64, 64))
prediction_image = image.img_to_array(prediction_image)
prediction_image = np.expand_dims(prediction_image, axis=0)

result = cnn_model.predict(prediction_image)

shoe_result = {
    0: 'Athletic',
    1: 'Clogs',
    2: 'Flats',
    3: 'Heels',
    4: 'Loafers'
}

shoe_personality_table = {
    0: 'The person is someone who’s very confident, very goal-oriented, and very organized.',
    1: 'The person is open spirited, and very outdoorsy. They love nature and the whole regenerative effect of being outdoors.',
    2: 'The person is focused, very modest, and generous.',
    3: 'The person really loves and values beauty.',
    4: 'The person is very responsible, very detail-driven, very much the person who manages all the details.'
}

personality = 0
for match, shoe in enumerate(result[0]):
    if match:
        personality = shoe
        break

print(shoe_personality_table[personality])

## Conclusion

Over the CNN model performs decently well. As shown in the CNN Model Training graph, the accuracy
and loss were diverging from the validation accuracy and validation loss. This suggests that the
model was over-fitting the data. Despite this, the CNN model correctly classified the single image
test shoe (my shoe).

To improve the CNN model's accuracy, a larger and more diverse dataset should be used. Due to the
time constraints of the exam, I was unable to do this.